# Query the live aircraft data from Opensky API

This notebook shows how to query live data from the Opensky API.
https://opensky-network.org/apidoc/

In [1]:
# Import the libraries that will be used in this notebook
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

## API configuration and query

In [2]:
# Save config information.
url = "https://opensky-network.org/api/states/all"

# Build partial query URL
query_url = f"{url}"

#  Perform a request for data
response = requests.get(query_url).json()

In [3]:
# Time of the query data
response["time"]

1607125240

In [4]:
# Amount of data point of the query data
len(response["states"])

4809

In [5]:
# One element of the array
response["states"][0]

['a25f32',
 'N252CV  ',
 'United States',
 1607125240,
 1607125240,
 -95.6393,
 45.5547,
 8229.6,
 False,
 156.65,
 60.49,
 0,
 None,
 8275.32,
 None,
 False,
 0]

In [6]:
# Map data to the dictorary
aircraft_live_data = [];
for i in range(len(response["states"])):
    data = {};
    # Test for null and strip spaces in case not null
    if response["states"][i][0]:
        data["icao24"] = response["states"][i][0].strip()
    else:
        data["icao24"] = response["states"][i][0]
    if response["states"][i][1]:
        data["callsign"] = response["states"][i][1].strip()
    else:
        data["callsign"] = response["states"][i][1]
    if response["states"][i][2]:
        data["origin_country"] = response["states"][i][2].strip()
    else:
        data["origin_country"] = response["states"][i][2]
    data["time_position"] = response["states"][i][3]
    data["last_contact"] = response["states"][i][4]
    data["longitude"] = response["states"][i][5]
    data["latitude"] = response["states"][i][6]
    data["baro_altitude"] = response["states"][i][7]
    data["on_ground"] = response["states"][i][8]
    data["velocity"] = response["states"][i][9]
    data["true_track"] = response["states"][i][10]
    data["vertical_rate"] = response["states"][i][11]
    data["sensors"] = response["states"][i][12]
    data["geo_altitude"] = response["states"][i][13]
    if response["states"][i][14]:
        data["squawk"] = response["states"][i][14].strip()
    else:
        data["squawk"] = response["states"][i][14]
    data["spi"] = response["states"][i][15]
    data["position_source"] = response["states"][i][16]
    
    aircraft_live_data.append(data)

In [7]:
aircraft_live_data[0]

{'icao24': 'a25f32',
 'callsign': 'N252CV',
 'origin_country': 'United States',
 'time_position': 1607125240,
 'last_contact': 1607125240,
 'longitude': -95.6393,
 'latitude': 45.5547,
 'baro_altitude': 8229.6,
 'on_ground': False,
 'velocity': 156.65,
 'true_track': 60.49,
 'vertical_rate': 0,
 'sensors': None,
 'geo_altitude': 8275.32,
 'squawk': None,
 'spi': False,
 'position_source': 0}

## Save data to MongoDB

In [14]:
import pymongo

# Use flask_pymongo to set up mongo connection
conn =  "mongodb://localhost:27017/project_3"
client =  pymongo.MongoClient(conn)

# identify the collection and drop any existing data for this demonstration
db = client.project_3
db.live_aircraft_data.drop()

# db.live_aircraft_data.insert_many(aircraft_live_data)


# query_result = list(db.mars_data.find())
# query_result = (db.mars_data.find())
query_result = (db.live_aircraft_data.find_one())
query_result

### Save data to MySQL

In [9]:
import datetime
from datetime import datetime
import mysql.connector
# import database_credentials as dbkeys
import os
import json

with open("/etc/config.json") as config_file:
    config = json.load(config_file)


# Variables:
database_name = "project_2"  # the name of the target database
table_name = "aircraft_data"



# Create a function to connect to the MYSQL server
def database_connect(hostname, username, password, database=database_name):
    mydb = mysql.connector.connect(
        host=hostname,
        user=username,
        passwd=password,
        database=database
    )
    return mydb



# Return an object containing the MYSQL connection
mydb = database_connect(
    config.get("MYSQL_HOSTNAME"),
    config.get("MYSQL_USERNAME"),
    config.get("MYSQL_PASSWORD")
    )
print(mydb)


# Create the cursor to manipute databases
my_cursor = mydb.cursor()
# Add unique constraint


my_cursor.execute(f"SELECT * FROM {database_name}.{table_name} ORDER BY id DESC LIMIT 1;")
for records in my_cursor:
    print(records)
    print(records[0])




# Create place holders records to insert into the table
sqlStuff = f"""INSERT INTO {table_name} (icao24, callsign,origin_country,time_position,last_contact,
                                        longitude,
                                        latitude,
                                        baro_altitude,
                                        on_ground,
                                        velocity,
                                        true_track,
                                        vertical_rate,
                                        sensors,
                                        geo_altitude,
                                        squawk,
                                        spi,
                                        position_source)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s) """

(4849, 'c00734', 'WJA126', 'Canada', 1607125079, 1607125079, -119.229, 49.7308, 12496.8, 0, 238.53, 70.47, -0.33, None, 12679.7, '2624', 0, 0)
4849


In [10]:
for  ii in range(len(aircraft_live_data)):
            record1 = (
                aircraft_live_data[ii]["icao24"],
                aircraft_live_data[ii]["callsign"],
                aircraft_live_data[ii]["origin_country"],
                aircraft_live_data[ii]["time_position"],
                aircraft_live_data[ii]["last_contact"],
                aircraft_live_data[ii]["longitude"],
                aircraft_live_data[ii]["latitude"],
                aircraft_live_data[ii]["baro_altitude"],
                aircraft_live_data[ii]["on_ground"],
                aircraft_live_data[ii]["velocity"],
                aircraft_live_data[ii]["true_track"],
                aircraft_live_data[ii]["vertical_rate"],
                aircraft_live_data[ii]["sensors"],
                aircraft_live_data[ii]["geo_altitude"],
                aircraft_live_data[ii]["squawk"],
                aircraft_live_data[ii]["spi"],
                aircraft_live_data[ii]["position_source"]
                )

#             print(record1)
                

            my_cursor.execute(sqlStuff, record1)

mydb.commit()

In [11]:
# aircraft_live_data[ii]

In [12]:
# my_cursor.execute(sqlStuff, record1)

In [13]:
# mydb.commit()

In [33]:
for i in range(len(response["states"])):
    aircraft_live_data = list(range(0,17))
    if response["states"][i][0]:
        aircraft_live_data[0] = response["states"][i][0].strip()
    else:
        aircraft_live_data[0] = response["states"][i][0]
    if response["states"][i][1]:
        aircraft_live_data[1] = response["states"][i][1].strip()
    else:
        aircraft_live_data[1] =  response["states"][i][1]
    if response["states"][i][2]:
        aircraft_live_data[2] =  response["states"][i][2].strip()
    else:
        aircraft_live_data[2] =  response["states"][i][2]
    aircraft_live_data[3] =  response["states"][i][3]
    aircraft_live_data[4] =  response["states"][i][4]
    aircraft_live_data[5] =  response["states"][i][5]
    aircraft_live_data[6] =  response["states"][i][6]
    aircraft_live_data[7] =  response["states"][i][7]
    aircraft_live_data[8] =  response["states"][i][8]
    aircraft_live_data[9] =  response["states"][i][9]
    aircraft_live_data[10] =  response["states"][i][10]
    aircraft_live_data[11] =  response["states"][i][11]
    aircraft_live_data[12] =  response["states"][i][12]
    aircraft_live_data[13] =  response["states"][i][13]
    if response["states"][i][14]:
        aircraft_live_data[14] =  response["states"][i][14].strip()
    else:
        aircraft_live_data[14] =  response["states"][i][14]
    aircraft_live_data[15] =  response["states"][i][15]
    aircraft_live_data[16]  = response["states"][i][16]

In [34]:
aircraft_live_data

['c00734',
 'WJA676',
 'Canada',
 1607135789,
 1607135789,
 -107.2557,
 50.764,
 11887.2,
 False,
 244.79,
 93.37,
 -0.65,
 None,
 11986.26,
 None,
 False,
 0]

In [37]:
(aircraft_live_data[0], 
 aircraft_live_data[1],
 aircraft_live_data[2],
 aircraft_live_data[3],
 aircraft_live_data[4],
 aircraft_live_data[5],
 aircraft_live_data[6],
 aircraft_live_data[7],
 aircraft_live_data[8],
 aircraft_live_data[9],
 aircraft_live_data[10],
 aircraft_live_data[11],
 aircraft_live_data[12],
 aircraft_live_data[13],
 aircraft_live_data[14],
 aircraft_live_data[15],
 aircraft_live_data[16]
)

('c00734',
 'WJA676',
 'Canada',
 1607135789,
 1607135789,
 -107.2557,
 50.764,
 11887.2,
 False,
 244.79,
 93.37,
 -0.65,
 None,
 11986.26,
 None,
 False,
 0)